# Totonto Neighbourhoods Notebook
## Scrubbing data from wikipedia
This Jupyter notebook explores and clusters Toronto neighbourhoods.
We first import all the tools we need.

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: \ 

Next we scrape the wikipedia page given for Neighbourhood names, and put them in a readable format.

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
page = BeautifulSoup(source,'lxml')

Viewing the page html, we find what part of the source code we wanted, and use the BeautifulSoup functions to get a list of the portions of html code representing rows in the table. We then remove the headers.

In [ ]:
table = page.find('table', class_='wikitable sortable')
tableLines = table.find_all('tr')
tableLines.pop(0)

We create an empty list, and, for each portion of html representing a row, extract the rows elements, and add them all to a list.

In [ ]:
listOfTableEntries = list()
for line in tableLines:
    lineElements = line.find_all('td')
    for element in lineElements:
        elementText = element.text
        listOfTableEntries.append(elementText)
    
    

Going through the list of all elemtns in the table, we reconstruct it into a list of list, each inside list representing a row in the table. We remove any rows with "not assigned" as a borough, and replace the neighbourhood with the borough name if the neighbourhood is 'Not assigned'. We also trim \n from the end of the neighbourhood names.

In [ ]:
listOfRows=list()
for i in range(0,867,3):
    if (listOfTableEntries[i+1] != 'Not assigned'):
        if (listOfTableEntries[i+2] == 'Not assigned\n'):
            listOfRows.append([listOfTableEntries[i],listOfTableEntries[i+1],listOfTableEntries[i+1]])
        else:
            listOfRows.append([listOfTableEntries[i],listOfTableEntries[i+1],listOfTableEntries[i+2][:-1]])


Using a while loop, we merge rows with the same PostalCode, getting a row ogf all neighbourhoods with the same PostalCose

In [ ]:
i = 0
while (i < len(listOfRows)-1):
    row = listOfRows[i]
    belowRow = listOfRows[i+1]
    if (row[0]==belowRow[0]):
        row[2]= row[2] + ', ' + belowRow[2]
        listOfRows.pop(i+1)
    else:
        i=i+1
        


We then place the whole thing into a dataframe, now that it has been processed as required.

In [ ]:
neighbourhoodTable = pd.DataFrame(listOfRows,columns = ['PostalCode','Borough','Neighbourhood'])

Looking at the head, we can see an example of the merging of PostalCodes in row indexes 2 and 3, and an exmaple of the replacement of 'Not assigned' as a neighbourhood with the borough name in row 4

In [ ]:
neighbourhoodTable.head()

In [ ]:
neighbourhoodTable.shape

## Adding in location data
First we shall get the lat/long data from the provided csv file.

In [ ]:
latLongData= pd.DataFrame.from_csv('https://cocl.us/Geospatial_data')

Lets check this data

In [ ]:
latLongData.head()

We use a table join, similar to those used in SQL, to combine the postal code lat/long data with the neighbourhood table we prepared above.

In [ ]:
neighbourhoodLocationTable = neighbourhoodTable.join(latLongData, on='PostalCode')

In [ ]:
neighbourhoodLocationTable.head()

Lets use geolocater to find the lat/long of Toronto

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Now we will use the Folium package to display the locations of the given nieghbourhoods on a map of toronto

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbourhoodLocationTable['Latitude'], neighbourhoodLocationTable['Longitude'], neighbourhoodLocationTable['Borough'], neighbourhoodLocationTable['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now lets do a bit of exploring of these neighbourhoods with FourSquare. We will first collect all of the nearby venues for each of these venues. YOU WILL NEED TO INPUT YOUR OWN FOURSQUARE CREDENTIALS.

In [ ]:
CLIENT_ID = 'KXMEYZ5FNM24P0FFVBOZ2Q0O2B0GDB1TVFKITKMYKSTLFRN5' # your Foursquare ID
CLIENT_SECRET = '54XYMKJPZ2XDIJAVVXBMQJJGRQIVRLZCSPX423LPHZRN5OIW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

We write a function that we can use to return for us all of the nearby venues to each PostalCode in our neighbourhoodLocationTable.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Lets retrieve those venues!

In [ ]:
torontoVenues=getNearbyVenues(neighbourhoodLocationTable['PostalCode'],neighbourhoodLocationTable['Latitude'],neighbourhoodLocationTable['Longitude'])
torontoVenues.head()

We'll use one hot enconding to assign numeric value and columns to each venue category

In [ ]:
# one hot encoding
torontoOneHot = pd.get_dummies(torontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add PostalCode column back to dataframe
torontoOneHot['PostalCode'] = torontoVenues['PostalCode'] 

# move PostalCode column to the first column
fixed_columns = [torontoOneHot.columns[-1]] + list(torontoOneHot.columns[:-1])
torontoOneHot = torontoOneHot[fixed_columns]

torontoOneHot.head()

We want group these together, and use mean to understand what proportion of nearby venues are of each category for each postal code.

In [ ]:
torontoGrouped = torontoOneHot.groupby('PostalCode').mean().reset_index()
torontoGrouped.head()

We will use the following method to show what type of venue is most frequently found near each Postal Code. This will help us find a nice area to live in!

In [ ]:
num_top_venues = 3

for pstCde in torontoGrouped['PostalCode']:
    print("----"+pstCde+"----")
    temp = torontoGrouped[torontoGrouped['PostalCode'] == pstCde].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Looking thorugh these, we find M2P has a bank, park and electronics store nearby. A perfect place for any up and coming data scientist who enjoys the outdoors! Lets find out what neighbourhoods have this postal code.

In [ ]:
neighbourhoodLocationTable.loc[neighbourhoodLocationTable['PostalCode']=='M2P']

So looks like York Mills West is the place to be!

## Clustering neighbourhoods
We'll cluster the neighbourhoods based upon their venues.

In [ ]:
# set number of clusters
kclusters = 5

torontoGroupedClustering = torontoGrouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Now we'll create a new dataframe that includes the clustering with the neighbourhood location data.

In [ ]:
torontoMerged = torontoGrouped

# add clustering labels
torontoMerged['Cluster Labels'] = kmeans.labels_

#Make it only contain the columns we want
torontoMerged = pd.concat([torontoMerged['PostalCode'],torontoMerged['Cluster Labels']],axis = 1)

#Merge in location data
torontoMerged = torontoMerged.join(latLongData, on='PostalCode')

torontoMerged.head() # check the last columns!

Fianlly, we can place the postal code on a map of Toronto, coloured by cluster.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontoMerged['Latitude'], torontoMerged['Longitude'], torontoMerged['PostalCode'], torontoMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

So we see the majority of neighbourhoods are in one cluster, with the remaining 4 clusters picking up a few others. Thanks for reading!